In [1]:
import wave
import numpy as np
import python_speech_features as ps
import os
import glob
import pickle
from scipy.io import wavfile

epsilon = 1e-5

def wgn(x, snr):
    snr = 10**(snr/10.0)
    xpower = np.sum(x**2)/len(x)
    npower = xpower / snr
    return np.random.randn(len(x)) * np.sqrt(npower)

def read_file(filename):
    file = wave.open(filename,'r')    
    audio_sample = wavfile.read(filename)  
    # print(audio_sample[1])  
    params = file.getparams()
    nchannels, sampwidth, framerate, wav_length = params[:4]
    str_data = file.readframes(wav_length)
    wavedata = np.fromstring(str_data, dtype = np.short)
    # librosa.load(wav_file_path + orig_wav_file, sr=sr)
    time = np.arange(0,wav_length) * (1.0/framerate)
    file.close()
    return audio_sample[1], time, framerate

def generate_label(control):
    label = 0
    if(control == 'cc'):
        label = 0
    elif(control == 'cd'):
        label = 1
    return label


In [2]:
filter_num = 40
_t = 300
rootdir = 'train/Full_wave_enhanced_audio'

train_label = []
train_data = []

In [3]:
# for control in os.listdir(rootdir):
    
#     sub_dir = rootdir + '/' + control
    
#     for sample in os.listdir(sub_dir):
#         data, time, rate = read_file(sub_dir+'/'+sample)
#         mel_spec = ps.logfbank(data,rate,nfilt = filter_num)
        
# #         print(data.shape[0])
#         time = mel_spec.shape[0] 
# #         print(mel_spec.shape)
# #         print(delta1.shape)
# #         print(delta2.shape)
        
#         length = time//256 * 257
        
#         print(length )
#         mel_spec = mel_spec[:length]
#         print(mel_spec.shape)
# #         delta1 = ps.delta(mel_spec, 2)
# #         delta2 = ps.delta(delta1, 2)
# #         print(time//256)
# #         for i in range(time//256):
# #             begin = 256*i
# #             end = begin + 256
# # #             if (time-begin<256): 
# # #                 end = time
# # #                 part = pad(mel_spec[begin:end,:],256,((0, 256),(before_N, after_N)),mode='constant')
# # #                 delta11 = delta1[begin:end,:]
# # #                 delta21 = delta2[begin:end,:]
                
                
# # #             else:
# #             part = mel_spec[begin:end,:]
# #             delta11 = delta1[begin:end,:]
# #             delta21 = delta2[begin:end,:]
                
            
            
# #             train_label.append(generate_label(control))
#         break  
          

In [4]:
train_label

[]

In [5]:
for control in os.listdir(rootdir):
    
    sub_dir = rootdir + '/' + control
    
    for sample in os.listdir(sub_dir):
        data, time, rate = read_file(sub_dir+'/'+sample)
        mel_spec = ps.logfbank(data,rate,nfilt = filter_num)
        time = mel_spec.shape[0] 
        mel_spec = mel_spec[:time//_t *_t,:]
        time = mel_spec.shape[0] 
        delta1 = ps.delta(mel_spec, 2)
        delta2 = ps.delta(delta1, 2)
        for i in range(time//_t):
            begin = _t*i
            end = begin + _t
            
            part = mel_spec[begin:end,:]
            delta11 = delta1[begin:end,:]
            delta21 = delta2[begin:end,:]
            
            _data = np.empty((_t,filter_num,3),dtype = np.float32)
            
            _data[:,:,0] = part
            _data[:,:,1] = delta11
            _data[:,:,2] = delta21
            
            train_label.append(generate_label(control))
            train_data.append(_data)

#         print(mel_spec[:time//256 *256,:].shape)
#         break
#     break


C:\Users\Asus\AppData\Local\Temp/ipykernel_4688/4117578347.py:24: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  wavedata = np.fromstring(str_data, dtype = np.short)


In [6]:
np.array(train_data).shape

(2541, 300, 40, 3)

In [7]:
np.array(train_label).shape

(2541,)

In [8]:
# from sklearn.model_selection import train_test_split
# train_size=0.8

# X_train, X_valid, y_train, y_valid = train_test_split(np.array(train_data),np.array(train_label), train_size=0.95)

In [9]:
rootdir = 'test'

test_label = []
test_data = []

for control in os.listdir(rootdir):
    
    sub_dir = rootdir + '/' + control
    print(os.listdir(sub_dir))
    
    for sample in os.listdir(sub_dir):
        print(sub_dir+'/'+sample)
        data, time, rate = read_file(sub_dir+'/'+sample)
        mel_spec = ps.logfbank(data,rate,nfilt = filter_num)
        time = mel_spec.shape[0] 
        mel_spec = mel_spec[:time//_t *_t,:]
        time = mel_spec.shape[0] 
        delta1 = ps.delta(mel_spec, 2)
        delta2 = ps.delta(delta1, 2)
        for i in range(time//_t):
            begin = _t*i
            end = begin + _t
            
            part = mel_spec[begin:end,:]
            delta11 = delta1[begin:end,:]
            delta21 = delta2[begin:end,:]
            
            _data = np.empty((_t,filter_num,3),dtype = np.float32)
            
            _data[:,:,0] = part
            _data[:,:,1] = delta11
            _data[:,:,2] = delta21
            
            print(control)
            test_label.append(generate_label(control))
            test_data.append(_data)

#         print(mel_spec[:time//256 *256,:].shape)
#         break
#     break


C:\Users\Asus\AppData\Local\Temp/ipykernel_4688/4117578347.py:24: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  wavedata = np.fromstring(str_data, dtype = np.short)


['S160.wav', 'S161.wav', 'S163.wav', 'S166.wav', 'S170.wav', 'S172.wav', 'S174.wav', 'S175.wav', 'S177.wav', 'S178.wav', 'S180.wav', 'S183.wav', 'S184.wav', 'S186.wav', 'S193.wav', 'S195.wav', 'S196.wav', 'S197.wav', 'S199.wav', 'S201.wav', 'S202.wav', 'S204.wav', 'S206.wav', 'S207.wav']
test/cc/S160.wav
cc
cc
cc
cc
cc
cc
cc
cc
test/cc/S161.wav


cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
test/cc/S163.wav
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
test/cc/S166.wav


cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
test/cc/S170.wav
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
test/cc/S172.wav


cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
cc
test/cc/S174.wav


...

cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
test/cd/S191.wav


cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
test/cd/S192.wav


cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
test/cd/S194.wav
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
test/cd/S198.wav


cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
test/cd/S200.wav
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
test/cd/S203.wav


cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
test/cd/S205.wav
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd
cd


In [10]:
np.array(test_data).shape, np.array(test_label).shape

((1242, 300, 40, 3), (1242,))

In [11]:
test_data = np.array(test_data)
test_label = np.array(test_label)
train_data = np.array(train_data)
train_label = np.array(train_label)

In [12]:
def randomize(a, b):
    # Generate the permutation index array.
    permutation = np.random.permutation(a.shape[0])
    # Shuffle the arrays by giving the permutation in the square brackets.
    shuffled_a = a[permutation]
    shuffled_b = b[permutation]
    return shuffled_a, shuffled_b


test_data,test_label = randomize(test_data,test_label)

In [13]:
train_data,train_label = randomize(train_data,train_label)

In [14]:
output = './adress_ADRNN_test.pkl'
f=open(output,'wb') 
pickle.dump((train_data,train_label,test_data,test_label),f)
f.close() 

In [15]:
train_label

array([1, 0, 1, ..., 0, 0, 1])